# We are scraping from reddit!!!

I feel like this is going to be a great project for me since I spend so much time lurking on reddit as it is. We have a couple things to go over:

1. This is a binary classification model
2. We pick two subreddits and try to grab about 1,000 posts from each one (but be careful of timings)
3. We then use NLP in our modeling to create a model that predicts which post came from where.


Possible subreddits:


In [59]:
#Let's grab some buddies

import requests
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
#Pick a subreddit to try this out
url = 'https://api.pushshift.io/reddit/search/submission'

In [62]:
#Created this code with the help of Derya
#Change before to the current time in order to get this to work on up to date data
before = 1616972862
df = pd.DataFrame()
total_df_lpt = pd.DataFrame()
# post_list = {}
for i in range(20):
    params_lpt = {
    'subreddit': 'lifeprotips',
    'size': 100,
    'before':before}
    res_lpt = requests.get(url, params_lpt)
    data_lpt = res_lpt.json()
    posts_lpt = data_lpt['data']
#    post_list[i] = posts
    df = pd.DataFrame(posts_lpt)
    total_df_lpt = pd.concat([df,total_df_lpt], axis=0, ignore_index=True)
    before = df['created_utc'].min()
    # before = posts[-1]['created_utc']
    time.sleep(60)
total_df_lpt.shape
# df = pd.DataFrame(post_list)

(2000, 74)

In [63]:
#We are in business!
total_df_lpt.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,author_flair_background_color,author_flair_text_color,banned_by,edited,author_cakeday,crosspost_parent,crosspost_parent_list,thumbnail_height,thumbnail_width,url_overridden_by_dest
0,[],False,levainletlive,None,[],None,text,t2_otior9k,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,RandomHuman01,None,[],None,text,t2_tpdoo,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,h3xadecimal2,None,[],None,text,t2_dyqx1k1,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,bethelmayflower,None,[],None,text,t2_b58sv,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,IFLE,None,[],None,text,t2_cmbb5,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
#Created this code with the help of Derya
#Change before to the current time in order to get this to work on up to date data
before = 1616972862
df = pd.DataFrame()
total_df_slpt = pd.DataFrame()
# post_list = {}
for i in range(20):
    params_slpt = {
    'subreddit': 'shittylifeprotips',
    'size': 100,
    'before':before}
    res_slpt = requests.get(url, params_slpt)
    data_slpt = res_slpt.json()
    posts_slpt = data_slpt['data']
#    post_list[i] = posts
    df = pd.DataFrame(posts_slpt)
    total_df_slpt = pd.concat([df,total_df_slpt], axis=0, ignore_index=True)
    before = df['created_utc'].min()
    # before = posts[-1]['created_utc']
    time.sleep(60)
total_df_slpt.shape
# df = pd.DataFrame(post_list)

(2000, 77)

# Exporting My Data to CSV to Continue in my Modeling Notebook

In [67]:
total_df_lpt.to_csv('./data/total_life_pro_tips.csv')
total_df_slpt.to_csv('./data/total_shitty_life_pro_tips.csv')

In [9]:
print(f'You have {len(posts_lpt)} Life Pro Tips!')
print(f'You have {len(posts_lpt)} Shitty Life Pro Tips!')

You have 100 Life Pro Tips!
You have 100 Shitty Life Pro Tips!


In [12]:
df_lpt = pd.DataFrame(posts_lpt)

In [13]:
df_lpt[['subreddit', 'selftext', 'title']].head()

,subreddit,selftext,title
0,LifeProTips,[removed],Mining Pi Crypto + Bee Crypto (Both Free and E...
1,LifeProTips,I’m almost always running quite late a bit mys...,LPT: If you happen to be running late for an a...
2,LifeProTips,Gonna put this out here because my mom got int...,"LPT: When parking, always back in to the parki..."
3,LifeProTips,,"LPT: When you move somewhere new, have a littl..."
4,LifeProTips,[removed],LPT: Help me make an impressive CV!


In [28]:
end_df_lpt = df_lpt[['subreddit', 'selftext', 'title']]

In [14]:
df_slpt = pd.DataFrame(posts_slpt)

In [15]:
end_df_slpt = df_slpt[['subreddit', 'selftext', 'title']].head()

,subreddit,selftext,title
0,ShittyLifeProTips,[deleted],Survival Tip . always works
1,ShittyLifeProTips,"Once you figure out which one is best, have on...",SLPT: not sure of the best parenting method? T...
2,ShittyLifeProTips,,SLPT: make alternating post between SLPT and L...
3,ShittyLifeProTips,,SLPT: Bored at home in quarantine? Stack your ...
4,ShittyLifeProTips,,SLPT This is the newest project of him?


In [26]:
end_df_slpt = df_slpt[['subreddit', 'selftext', 'title']]

In [27]:
end_df_slpt.head()

,subreddit,selftext,title
0,ShittyLifeProTips,[deleted],Survival Tip . always works
1,ShittyLifeProTips,"Once you figure out which one is best, have on...",SLPT: not sure of the best parenting method? T...
2,ShittyLifeProTips,,SLPT: make alternating post between SLPT and L...
3,ShittyLifeProTips,,SLPT: Bored at home in quarantine? Stack your ...
4,ShittyLifeProTips,,SLPT This is the newest project of him?
